In [5]:
from stable_baselines3.ppo import CnnPolicy
from stable_baselines3 import PPO
from pettingzoo.butterfly import pistonball_v6
import supersuit as ss

In [6]:
env = pistonball_v6.parallel_env(n_pistons=20, time_penalty=-0.1, continuous=True, random_drop=True, random_rotate=True, ball_mass=0.75, ball_friction=0.3, ball_elasticity=1.5, max_cycles=125)

In [7]:
env = ss.color_reduction_v0(env, mode='B')

In [9]:
env = ss.resize_v1(env, x_size=84, y_size=84)

In [10]:
env = ss.frame_stack_v1(env, 3)

In [11]:
env = ss.pettingzoo_env_to_vec_env_v1(env)

In [12]:
env = ss.concat_vec_envs_v1(env, 8, num_cpus=4, base_class='stable_baselines3')

In [13]:
model = PPO(CnnPolicy, env, verbose=3, gamma=0.95, n_steps=256, ent_coef=0.0905168, learning_rate=0.00062211, vf_coef=0.042202, max_grad_norm=0.9, gae_lambda=0.99, n_epochs=5, clip_range=0.3, batch_size=256)
model.learn(total_timesteps=2000000)
model.save("policy")

Using cpu device
Wrapping the env in a VecTransposeImage.
------------------------------
| time/              |       |
|    fps             | 562   |
|    iterations      | 1     |
|    time_elapsed    | 72    |
|    total_timesteps | 40960 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 230         |
|    iterations           | 2           |
|    time_elapsed         | 356         |
|    total_timesteps      | 81920       |
| train/                  |             |
|    approx_kl            | 0.011083754 |
|    clip_fraction        | 0.0379      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.5        |
|    explained_variance   | -0.00276    |
|    learning_rate        | 0.000622    |
|    loss                 | 0.708       |
|    n_updates            | 5           |
|    policy_gradient_loss | 0.00326     |
|    std                  | 1.11        |
|    value_

In [ ]:
env.reset()
for agent in env.agent_iter():
   obs, reward, done, info = env.last()
   act = model.predict(obs, deterministic=True)[0] if not done else None
   env.step(act)
   env.render()